必要配置项
 - `ROOTPATH` 网盘的根目录路径，一般无需更改
 - `ModelPath` 模型路径
 - `PindingPath` 待处理图片路径
 - `FinishPath` 图片完成存放路径
 - `ModelName`  模型名称
 - `Tile`  有6种模式，数字越大显存需求越低，相对地可能会小幅降低推理速度 **{0, 1, 2, 3, 4, auto}**
 - `Amplification` 放大倍数，需要与模型匹配



In [5]:
ROOTPATH="/content/gdrive/MyDrive/"
ModelPath=ROOTPATH+"Real-CUGAN/model/"
PendingPath=ROOTPATH+"Real-CUGAN/pending/"
FinishPath=ROOTPATH+"Real-CUGAN/finish/"
ModelName="up2x-latest-no-denoise.pth"
Tile=4
Amplification=ModelName[2]

In [6]:
#初始化环境并挂载谷歌云盘
!pip install torch opencv-python
!git clone https://github.com/bilibili/ailab.git
from google.colab import drive
drive.mount('/content/gdrive')

fatal: destination path 'ailab' already exists and is not an empty directory.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
import os
fileNames = os.listdir(PendingPath)
print("目前待处理图片")
for i in fileNames:
  print("\t"+i)
if (not os.path.isfile(ModelPath+ModelName)):
  print("警告：当前选择的模型不存在")
  fileNames = os.listdir(ModelPath)
  print("目前已有模型")
  for i in fileNames:
    print("\t"+i)

目前待处理图片


In [8]:
import sys
sys.path.append("/content/ailab/Real-CUGAN")
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn as nn
from torch.nn import functional as F
import os,sys,cv2
import numpy as np
from upcunet_v3 import RealWaifuUpScaler
from time import time as ttime  
fileNames = os.listdir(PendingPath)
upscaler = RealWaifuUpScaler(Amplification, ModelPath+ModelName, half=True, device="cuda:0")
t0 = ttime()
for i in fileNames:
  torch.cuda.empty_cache()
  try:
    img = cv2.imread(PendingPath+i)[:, :, [2, 1, 0]]
    result = upscaler(img,tile_mode=2)
    cv2.imwrite(FinishPath+i,result[:, :, ::-1])
  except RuntimeError as e:
    print (i+"失败")
    print (e)
  else:
    print(i+"已完成")
    os.remove(PendingPath+i)
t1 = ttime()
print("done", t1 - t0)

done 0.0001957416534423828
